In [1]:
#!pip install ckiptagger
!pip install ckip-transformers

In [2]:
# 覆寫print
output = open('output.txt', 'w', encoding='utf-8')
def p(*args, **kwargs):
    print(*args, **kwargs) # 終端機輸出
    print(*args, **kwargs, file=output, flush=True) # 寫入檔案

In [3]:
import os
os.environ["TQDM_DISABLE"] = "True"

import json
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger#, CkipNerChunker
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import csv
import torch
import pandas as pd
from tqdm import tqdm
import fasttext
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
p(device)

cuda


In [4]:
# 讀取 JSON 檔案
with open('/kaggle/input/ir-final-project/ithome_news.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [5]:
# 取得 content 欄位
text = [item["content"] for item in data if "content" in item]
_id = [item['_id'].split('/')[-1] for item in data if '_id' in item]

In [6]:
# 初始化 CKIP 斷詞

# 使用 ckip transformer
ws_driver = CkipWordSegmenter(model="bert-base", device=device)
pos_driver = CkipPosTagger(model="bert-base", device=device)

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/407M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.86k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/407M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
ws = ws_driver(text)

In [8]:
print(pos_driver([ws[0]]))

[['P', 'Nd', 'Nd', 'VHC', 'COMMACATEGORY', 'Neu', 'Nd', 'Neu', 'Na', 'Na', 'Na', 'Ng', 'COMMACATEGORY', 'Nc', 'VH', 'Nc', 'DE', 'Na', 'Na', 'VG', 'A', 'Na', 'COMMACATEGORY', 'Cbb', 'P', 'Nd', 'Na', 'VG', 'VH', 'Na', 'DE', 'FW', 'COMMACATEGORY', 'Nep', 'Na', 'Na', 'Na', 'D', 'D', 'VCL', 'Na', 'Na', 'PERIODCATEGORY', 'P', 'Nc', 'Na', 'Na', 'Na', 'COMMACATEGORY', 'P', 'Nd', 'Nd', 'Nd', 'PARENTHESISCATEGORY', 'Nd', 'PARENTHESISCATEGORY', 'Ng', 'COMMACATEGORY', 'V_2', 'VJ', 'Neu', 'Nf', 'VH', 'Nc', 'VE', 'Na', 'Na', 'PERIODCATEGORY', 'VH', 'VK', 'DE', 'SHI', 'COMMACATEGORY', 'Nep', 'Nf', 'Na', 'P', 'VC', 'DE', 'Na', 'COMMACATEGORY', 'V_2', 'Neqa', 'Nc', 'D', 'VG', 'VC', 'Na', 'PARENTHESISCATEGORY', 'FW', 'PARENTHESISCATEGORY', 'Nc', 'COMMACATEGORY', 'VK', 'Nb', 'Na', 'PAUSECATEGORY', 'Nb', 'Na', 'PAUSECATEGORY', 'Nb', 'Na', 'COMMACATEGORY', 'Neqa', 'D', 'V_2', 'Na', 'PAUSECATEGORY', 'Na', 'Na', 'PAUSECATEGORY', 'FW', 'Na', 'Caa', 'Na', 'Na', 'Cab', 'Na', 'PERIODCATEGORY', 'FW', 'Nb', 'Na', 

In [9]:
#pos = pos_driver(ws, batch_size=32)
pos = []
for t in tqdm(ws, desc="pos index", total = len(ws), disable=False):
    pos.append(pos_driver([t]))

pos index: 100%|██████████| 3793/3793 [11:09<00:00,  5.67it/s]


In [10]:
print(pos[0][0])

['P', 'Nd', 'Nd', 'VHC', 'COMMACATEGORY', 'Neu', 'Nd', 'Neu', 'Na', 'Na', 'Na', 'Ng', 'COMMACATEGORY', 'Nc', 'VH', 'Nc', 'DE', 'Na', 'Na', 'VG', 'A', 'Na', 'COMMACATEGORY', 'Cbb', 'P', 'Nd', 'Na', 'VG', 'VH', 'Na', 'DE', 'FW', 'COMMACATEGORY', 'Nep', 'Na', 'Na', 'Na', 'D', 'D', 'VCL', 'Na', 'Na', 'PERIODCATEGORY', 'P', 'Nc', 'Na', 'Na', 'Na', 'COMMACATEGORY', 'P', 'Nd', 'Nd', 'Nd', 'PARENTHESISCATEGORY', 'Nd', 'PARENTHESISCATEGORY', 'Ng', 'COMMACATEGORY', 'V_2', 'VJ', 'Neu', 'Nf', 'VH', 'Nc', 'VE', 'Na', 'Na', 'PERIODCATEGORY', 'VH', 'VK', 'DE', 'SHI', 'COMMACATEGORY', 'Nep', 'Nf', 'Na', 'P', 'VC', 'DE', 'Na', 'COMMACATEGORY', 'V_2', 'Neqa', 'Nc', 'D', 'VG', 'VC', 'Na', 'PARENTHESISCATEGORY', 'FW', 'PARENTHESISCATEGORY', 'Nc', 'COMMACATEGORY', 'VK', 'Nb', 'Na', 'PAUSECATEGORY', 'Nb', 'Na', 'PAUSECATEGORY', 'Nb', 'Na', 'COMMACATEGORY', 'Neqa', 'D', 'V_2', 'Na', 'PAUSECATEGORY', 'Na', 'Na', 'PAUSECATEGORY', 'FW', 'Na', 'Caa', 'Na', 'Na', 'Cab', 'Na', 'PERIODCATEGORY', 'FW', 'Nb', 'Na', '

In [11]:
def clean(sentence_ws, sentence_pos):
    words = []
    #stay_pos = ['A', 'Na', 'Nb', 'Nc', 'Nd', 'VA', 'VAC', 'VB', 'VC', 'VCL', 'VD', 'VF', 'VE', 'VG', 'VH', 'VHC', 'VI',
    #            'VJ', 'VK', 'VL']  # 保留詞性
    stay_pos = ['A', 'Na', 'Nb', 'Nc', 'Nd']  # 保留詞性
    for word_ws, word_pos in zip(sentence_ws, sentence_pos):
        is_not_stop_pos = word_pos in stay_pos
        # 組成串列
        if is_not_stop_pos:
            words.append(f"{word_ws}")
    return words

In [12]:
all_words = [] # [每篇文章出現過的詞集合]
with tqdm(total = len(ws), desc='clear ', disable=False) as pbar:
    for sentence_ws, sentence_pos in zip(ws, pos):
        words = clean(sentence_ws, sentence_pos[0])
        all_words.extend(list(set(words)))
        pbar.update(1)

word_freq = {word: all_words.count(word) for word in set(all_words)}
sorted_keywords = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

clear : 100%|██████████| 3793/3793 [00:00<00:00, 9569.14it/s]


In [13]:
print(len(all_words))

340588


In [14]:
p("\n\n基於詞性過濾的關鍵詞：")
for i in range(20):
    p(sorted_keywords[i])



基於詞性過濾的關鍵詞：
('者', 2669)
('資料', 2180)
('系統', 2086)
('程式', 1864)
('人員', 1755)
('業者', 1726)
('公司', 1721)
('功能', 1699)
('應用', 1688)
('網路', 1677)
('用戶', 1672)
('平臺', 1629)
('資安', 1570)
('軟體', 1537)
('工具', 1514)
('資訊', 1477)
('今年', 1474)
('目前', 1427)
('技術', 1422)
('企業', 1397)


In [15]:
# deprecated

# 方式 2：基於 TF-IDF 提取關鍵詞
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([' '.join(i) for i in ws])
feature_names = vectorizer.get_feature_names_out()

# create pandas DataFrame with tf-idf scores: Term-Document Matrix
df_tf_idf = pd.DataFrame(tfidf_matrix.T.todense(), index = feature_names, columns = _id)
some_df = df_tf_idf.iloc[-20, -20]
p(some_df)

0.0


In [16]:
# 方式 2：基於 TF-IDF 提取關鍵詞
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([' '.join(i) for i in ws])
feature_names = vectorizer.get_feature_names_out()


In [17]:
for i in range(10):
    tfidf_scores = dict(zip(feature_names, tfidf_matrix.toarray()[i]))
    sorted_tfidf = sorted(tfidf_scores.items(), key=lambda x: x[1], reverse=True)
    
    p("\n\n基於 TF-IDF 的關鍵詞：")
    for j in range(20):
        p(sorted_tfidf[j])
    p('')



基於 TF-IDF 的關鍵詞：
('漏洞', 0.32976056561503686)
('deepseek', 0.24531488425417922)
('cve', 0.23890466324272136)
('攻擊', 0.1577532299871617)
('資安', 0.15038372616758272)
('電路板', 0.13672655340952236)
('遭受', 0.12639031769765496)
('2月', 0.12392561957559596)
('2025', 0.12149014309988335)
('政府', 0.1147602423656468)
('0411', 0.11435975862263734)
('春節', 0.10555191707684408)
('修補', 0.09964223914079935)
('執法', 0.09923202121132116)
('利用', 0.09761736106346627)
('工具', 0.09580283087173776)
('motw', 0.09516527359063638)
('軟體', 0.09505022473894563)
('zip', 0.09323447495066718)
('部份', 0.09000800718299201)



基於 TF-IDF 的關鍵詞：
('漏洞', 0.30779772392503574)
('攻擊', 0.2273659004204286)
('駭客', 0.22714819149921825)
('群暉', 0.20218386918033285)
('軟體', 0.1774393052646834)
('套件', 0.17044824640139106)
('惡意', 0.1614578896746931)
('code', 0.14800496225690737)
('網站', 0.13716465427013472)
('veeam', 0.1332193842025974)
('press', 0.1300199223848046)
('備份', 0.12231842729895127)
('word', 0.11878208866293406)
('vs', 0.118113083386

In [18]:
model = fasttext.load_model('/kaggle/input/fasttext-chinese/other/default/1/cc.zh.300.bin')

In [19]:
vectors = []
for t in tqdm(text, disable=False):
    vector = model.get_sentence_vector(t)
    vectors.append(vector)

100%|██████████| 3793/3793 [00:03<00:00, 1084.45it/s]


In [20]:
import numpy as np
#vecetors = np.array(vectors)

In [21]:
n_clusters = 50  # 群數
kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=42)
labels = kmeans.fit_predict(vectors)

In [22]:
# 5. 把分群結果寫進每篇資料，並且插在 date 後面
new_data = []
insert_after_key = 'date'
new_key = 'cluster'
output_file = f"{n_clusters}clusters_ithome_news.csv"

for item, label in zip(data, labels):
    new_item = {}

    # 更新 _id 欄位，提取 URL 中的數字
    new_item['_id'] = item['_id'].split('/')[-1]  # 更新 _id 為數字部分
    for k, v in item.items():
        if k != '_id':  # 避免重新賦值 _id
            new_item[k] = v
        if k == insert_after_key:
            new_item[new_key] = int(label)  # 寫入群組 (轉成 int 比較好看)
    new_data.append(new_item)

# 6. 輸出為 CSV 檔案
with open(output_file, 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["_id", "url", "date", "cluster", "title", "content"])
    writer.writeheader()
    writer.writerows(new_data)

print(f"✅ 完成！加上 cluster 後已存到 {output_file}")

✅ 完成！加上 cluster 後已存到 50clusters_ithome_news.csv


In [ ]:
label_keywords = {
    "加密貨幣": "加密貨幣 比特幣 區塊鏈 crypto",
    "資安漏洞": "資安 漏洞 密碼 攻擊 滲透 駭客 金鑰 CVE 惡意 零時差",
    "伺服器相容性": "伺服器 相容性 相容 協定 架構",
    "AI": "人工智慧 AI 機器學習 模型",
    "Cloud": "雲端 Cloud 雲端儲存 雲服務",
    "程式語言": "Rust Python C++",
    "數位內容透明度": "數位內容 透明度 內容審查 媒體",
    "傳輸頻寬": "傳輸 頻寬 數據 傳送 協定",
    "法律": "法律 刑法 民法 條文 立法",
    "遊戲": "遊戲 MyCard 玩家 電玩"
}


from sklearn.metrics.pairwise import cosine_similarity

label_vectors = {
    label: np.mean([model.get_word_vector(w) for w in keywords.split()], axis=0)
    for label, keywords in label_keywords.items()
}

In [25]:
threshold = 0.6

def classify_article(title, content):
    text_vec = np.mean([model.get_sentence_vector(title), model.get_sentence_vector(content)], axis=0)
    sims = {label: cosine_similarity([text_vec], [vec])[0][0] for label, vec in label_vectors.items()}\

    filtered_labels = [
        (label, score)
        for label, score in sims.items()
        #if score > threshold
    ]

    filtered_labels.sort(key=lambda x: x[1], reverse=True)
    return filtered_labels  # 回傳所有超過門檻的類別與分數


    #best_label = max(sims, key=sims.get)
    #best_score = sims[best_label]
    #return best_label, best_score  # 同時回傳分類與相似度分數

In [26]:
with open("classified_articles.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["_id", "classified_labels", "title", "content"])  # 移除單一 score 欄，因為現在是多個

    for article in data:
        results = classify_article(article['title'], article['content'])  # 回傳多個 (label, score)

        # 格式化：資安漏洞(0.81);AI(0.65)
        if results:
            label_score_str = ";".join([f"{label}({score:.2f})" for label, score in results])
        else:
            label_score_str = ""  # 沒有任何類別符合門檻

        writer.writerow([
            article['_id'],
            label_score_str,
            article['title'],
            article['content']
        ])

#with open("classified_articles.csv", "w", encoding="big5", newline="") as f:
#    writer = csv.writer(f)
#    writer.writerow(["_id", "classified_label", 'score', "title", 'content'])
#    for article in data:
#        label, score = classify_article(article['title'],  article['content'])
#        writer.writerow([article['_id'], label, score, article['title'],  article['content']])